In [14]:
import csv
import gzip
import pandas as pd
import h5py

# df_p1load_1week = pd.read_csv('load.csv.gz', compression='gzip', header=0, nrows=168)
# print(df_p1load_1week["p1"])

#df_p1_census = pd.read_csv('co-est2021-alldata.csv', header=0, usecols=['STNAME','CTYNAME','POPESTIMATE2021','STATE','COUNTY'], nrows=3195, encoding='latin-1')
#print(df_p1_census)

#df_p1_counties = pd.read_excel('county_reeds_corrected0310.xlsx', header=0, usecols=['NAME','STATE_NAME','FIPS','PCA_REG'])
#print(df_p1_counties)

# filename = "Reeds_Spatialflex\Clean2035clip1pct_load_hourly.h5"
# df_drdec_baseline = h5py.File(filename, 'r')
# df_drdec_baseline.keys()
# #df_drdec_baseline.close()

def read_file(filename, index_columns=1):
    """
    Read input file of various types (for backwards-compatibility)
    """
    # Try reading a .h5 file written by pandas
    try:
        df = pd.read_hdf(filename+'.h5')
    # Try reading a .h5 file written by h5py
    except (ValueError, TypeError, FileNotFoundError, OSError):
        try:
            with h5py.File(filename+'.h5', 'r') as f:
                keys = list(f)
                datakey = 'data' if 'data' in keys else ('cf' if 'cf' in keys else 'load')
                ### If none of these keys work, we're dealing with EER-formatted load
                if datakey not in keys:
                    years = [int(y) for y in keys if y != 'columns']
                    df = pd.concat(
                        {y: pd.DataFrame(f[str(y)][...]) for y in years},
                        axis=0)
                    df.index = df.index.rename(['year','hour'])
                else:
                    df = pd.DataFrame(f[datakey][:])
                    df.index = pd.Series(f['index']).values
                df.columns = pd.Series(f['columns']).map(
                    lambda x: x if type(x) is str else x.decode('utf-8')).values
        # Fall back to .csv.gz
        except (FileNotFoundError, OSError):
            df = pd.read_csv(
                filename+'.csv.gz', index_col=list(range(index_columns)),
                float_precision='round_trip',
            )

    return df


df_p1load_1week = read_file('C:\\Users\\akarmaka\\Documents\\GitHub\\ReEDS-2.0\\inputs\\loaddata\\RTS_load_hourly',
            index_columns=2)


# df_drdec_baseline = read_file('Clean2035clip1pct_load_hourly',
#             index_columns=2)



#
# df_drdec_baseline = pd.read_hdf('Reeds_Spatialflex\Clean2035clip1pct_load_hourly.h5', header=0)
df_p1load_1week

z1           z2           z3
year hour                                       
2010 0      891.725586   998.238098  1131.279297
     1      892.363892   980.368896  1079.449463
     2      906.725952   977.526062  1055.370972
     3      938.003357   976.307739  1041.087158
     4     1031.835693   994.989136  1053.738525
...                ...          ...          ...
2050 8755  1760.548462  1868.382080  2354.524902
     8756  1711.129517  1817.591187  2275.534424
     8757  1585.206421  1757.125732  2167.985840
     8758  1466.886108  1675.497314  1988.130493
     8759  1389.431519  1589.031738  1821.035156

[359160 rows x 3 columns]

In [15]:
df_p1load_1week_2 = df_p1load_1week.transpose()
df_p1load_1week_2

year         2010                                                      \
hour         0            1            2            3            4      
z1     891.725586   892.363892   906.725952   938.003357  1031.835693   
z2     998.238098   980.368896   977.526062   976.307739   994.989136   
z3    1131.279297  1079.449463  1055.370972  1041.087158  1053.738525   

year                                                                   ...  \
hour         5            6            7            8            9     ...   
z1    1164.924194  1219.500122  1181.839600  1115.774048  1072.049438  ...   
z2    1046.566162  1116.418457  1105.047119  1090.020752  1094.894165  ...   
z3    1112.914429  1238.612183  1322.274536  1417.772217  1483.477783  ...   

year         2050                                                      \
hour         8750         8751         8752         8753         8754   
z1    1498.723267  1533.886719  1667.412842  1816.144775  1801.414062   
z2    1536.426880  1555.171143  1682.753174  1857.498291  1883.498413   
z3    2089.602783  2095.678955  2131.528564  2380.652588  2398.881104   

year                                                                   
hour         8755         8756         8757         8758         8759  
z1    1760.548462  1711.129517  1585.206421  1466.886108  1389.431519  
z2    1868.382080  1817.591187  1757.125732  1675.497314  1589.031738  
z3    2354.524902  2275.534424  2167.985840  1988.130493  1821.035156  

[3 rows x 359160 columns]

In [19]:
df_p1load_1week_2.index

Index(['z1', 'z2', 'z3'], dtype='object')